In [1]:
import argparse
import torch
import importlib
import pipeline_flux_rf_inversion
importlib.reload(pipeline_flux_rf_inversion)
from pipeline_flux_rf_inversion import FluxRFInversionPipeline
from diffusers import FluxImg2ImgPipeline, AutoencoderKL
from diffusers.utils import load_image
from diffusers.image_processor import VaeImageProcessor
from transformers import T5EncoderModel, T5TokenizerFast, CLIPTokenizer, CLIPTextModel
import torch
import gc
from IPython.display import Image
import os

from PIL import Image

ckpt_id = "black-forest-labs/FLUX.1-dev"

/opt/conda/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
text_encoder = CLIPTextModel.from_pretrained(
    ckpt_id, subfolder="text_encoder", torch_dtype=torch.bfloat16
)
text_encoder_2 = T5EncoderModel.from_pretrained(
    ckpt_id, subfolder="text_encoder_2", torch_dtype=torch.bfloat16
)
tokenizer = CLIPTokenizer.from_pretrained(ckpt_id, subfolder="tokenizer")
tokenizer_2 = T5TokenizerFast.from_pretrained(ckpt_id, subfolder="tokenizer_2")

pipeline_encoder = FluxRFInversionPipeline.from_pretrained(
    ckpt_id,
    text_encoder=text_encoder,
    text_encoder_2=text_encoder_2,
    tokenizer=tokenizer,
    tokenizer_2=tokenizer_2,
    transformer=None,
    vae=None,
).to("cuda:0")

vae = AutoencoderKL.from_pretrained("black-forest-labs/FLUX.1-schnell", revision="refs/pr/1", subfolder="vae", torch_dtype=torch.bfloat16).to(
    "cuda:0"
)

Loading checkpoint shards: 100%|███████████████████████████████████████████████| 2/2 [00:00<00:00,  8.77it/s]
You set `add_prefix_space`. The tokenizer needs to be converted from the slow tokenizers
Loading pipeline components...: 100%|█████████████████████████████████████████| 5/5 [00:00<00:00, 609.90it/s]


In [3]:

pipeline = FluxRFInversionPipeline.from_pretrained(
    #"black-forest-labs/FLUX.1-dev",
    ckpt_id,
    text_encoder=None,
    text_encoder_2=None,
    tokenizer=None,
    tokenizer_2=None,
    vae=None,
    torch_dtype=torch.bfloat16,
    generator=torch.Generator("cuda:1").manual_seed(0)
).to("cuda:1")


Keyword arguments {'generator': <torch._C.Generator object at 0x724ec5f8b550>} are not expected by FluxRFInversionPipeline and will be ignored.
Loading pipeline components...: 100%|██████████████████████████████████████████| 2/2 [00:00<00:00,  3.80it/s]


In [9]:
init_image = Image.open("panda.png").resize((1024, 1024))
prompt = "a panda wearing a crown"
prompt_2 = prompt
gamma = 0.5
eta = 0.9
start_timestep = 0
stop_timestep = 12
num_inference_steps = 28
strength = 0.95
guidance_scale = 3.5
output = "stop12.jpg"

# split basename and extension
save_base, ext = output.rsplit(".", 1)

kwargs = {"gamma": gamma, "eta": eta, "start_timestep": start_timestep, "stop_timestep": stop_timestep} 


print("Encoding prompts.")
prompt_embeds, pooled_prompt_embeds, text_ids = pipeline_encoder.encode_prompt(
    prompt=prompt, prompt_2=None, max_sequence_length=256
)

print("Encoding null prompts.")
null_prompt_embeds, null_pooled_prompt_embeds, null_text_ids = pipeline_encoder.encode_prompt(
    prompt="", prompt_2="", max_sequence_length=256
)



Encoding prompts.
Encoding null prompts.


In [10]:

images = pipeline(
    prompt_embeds=prompt_embeds.to("cuda:1"),
    pooled_prompt_embeds=pooled_prompt_embeds.to("cuda:1"),
    null_prompt_embeds=null_prompt_embeds.to("cuda:1"),
    null_pooled_prompt_embeds=null_pooled_prompt_embeds.to("cuda:1"),
    image=init_image,
    num_inference_steps=num_inference_steps,
    strength=strength,
    guidance_scale=guidance_scale,
    vae=vae,
    **kwargs,
).images[0]

images.save(f"{save_base}.{ext}")
print(f"Output image saved as {save_base}.{ext}")

100%|████████████████████████████████████████████████████████████████████████| 27/27 [00:30<00:00,  1.11s/it]


Output image saved as stop12.jpg
